In [ ]:
import requests
import pandas as pd
from IPython.display import clear_output, display
import datetime

In [ ]:

URL_2023 = "https://results.xacte.com/json/agegroup?categoryId=9756&distanceId=18838&eventId=2522&limit=1000&offset={}&subeventId=6279"
URL_2022 = "https://results.xacte.com/json/agegroup?categoryId=9490&distanceId=18352&eventId=2462&limit=1000&offset={}&subeventId=6140"

URL = URL_2022
YEAR=2022

In [ ]:
pg_index=1

count=0

df_final=None

while True:
    res = requests.get(URL.format(count))
    data=res.json()
    
    if 'aaData' not in data or len(data['aaData'])==0:
        break
    count+=len(data['aaData'])
    
    clear_output()
    display(f"{pg_index} - {count}")

    df=pd.DataFrame(data['aaData'])

    if df_final is None:
        df_final=df
    else:
        df_final=pd.concat([df_final, df])

    pg_index+=1



In [ ]:
df_final.reset_index(inplace=True)

In [ ]:
df_final['time']= df_final.apply(lambda row: (datetime.datetime(1970,1,1) + datetime.timedelta(milliseconds=row['splits'][str(row['distanceId'])]['elapsed'])).time().isoformat(timespec='seconds'), axis=1)


In [ ]:
#res = requests.post(URL, json=qry, headers=headers)
df_final.to_csv(f"phila_{YEAR}_full.csv")

In [ ]:
df_final[['country','sex','age','time']].to_csv(f"phila_{YEAR}.csv")

In [ ]:
df_final[['age','sex']].groupby('sex', dropna=False).count()

In [ ]:
df_final[df_final.sex.isna()]